Predict the closest block that will get the driver the highest tips based on the time of the day and day of the week.

In [1]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline

In [2]:
data_2012_01=pd.read_csv("/Users/RAFAELLICHEN/Downloads/column_deleted_data_2012/yellow_tripdata_2012-01.csv")
data_2012_01=data_2012_01.drop("Unnamed: 0",1)
data_2012_01=data_2012_01.drop("passenger_count",1)
data_2012_01.head()

,pickup_datetime,dropoff_datetime,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,tip_amount,total_amount
0,2012-01-10 23:55:50,2012-01-11 00:03:39,1.7,-73.994693,40.725031,-73.975951,40.730781,1.00,8.90
1,2012-01-11 19:18:25,2012-01-11 19:26:10,1.1,-73.987955,40.752947,-73.994532,40.761040,0.00,7.60
2,2012-01-11 19:19:19,2012-01-11 19:48:15,18.0,-73.783093,40.648550,-73.996134,40.747624,10.06,60.36
3,2012-01-11 19:19:21,2012-01-11 19:27:00,1.7,-73.967515,40.758454,-73.956583,40.779903,1.00,9.40
4,2012-01-11 14:38:15,2012-01-11 14:43:51,1.2,-74.011315,40.711449,-74.002872,40.728131,0.00,6.20


In [3]:
jan_time=pd.to_datetime(pd.Series(data_2012_01["pickup_datetime"]))
jan_time_list=[]
for index in jan_time:
    jan_time_list.append(index.dayofweek)
data_2012_01["weekday"]=jan_time_list
data_2012_01.head()

,pickup_datetime,dropoff_datetime,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,tip_amount,total_amount,weekday
0,2012-01-10 23:55:50,2012-01-11 00:03:39,1.7,-73.994693,40.725031,-73.975951,40.730781,1.00,8.90,1
1,2012-01-11 19:18:25,2012-01-11 19:26:10,1.1,-73.987955,40.752947,-73.994532,40.761040,0.00,7.60,2
2,2012-01-11 19:19:19,2012-01-11 19:48:15,18.0,-73.783093,40.648550,-73.996134,40.747624,10.06,60.36,2
3,2012-01-11 19:19:21,2012-01-11 19:27:00,1.7,-73.967515,40.758454,-73.956583,40.779903,1.00,9.40,2
4,2012-01-11 14:38:15,2012-01-11 14:43:51,1.2,-74.011315,40.711449,-74.002872,40.728131,0.00,6.20,2


In [41]:
jan_2012_mon=pd.DataFrame(data_2012_01)
jan_2012_mon=jan_2012_mon.loc[jan_2012_mon["weekday"]==0]
jan_2012_mon=jan_2012_mon.reset_index(drop=True)
print(jan_2012_mon.shape)

(2098157, 10)


In [42]:
jan_2012_mon=jan_2012_mon[jan_2012_mon.pickup_longitude!=0]
jan_2012_mon=jan_2012_mon[jan_2012_mon.pickup_latitude!=0]
jan_2012_mon=jan_2012_mon[jan_2012_mon.pickup_latitude>40]
jan_2012_mon=jan_2012_mon[jan_2012_mon.pickup_latitude<41]
jan_2012_mon=jan_2012_mon[jan_2012_mon.pickup_longitude>-74.5]
jan_2012_mon=jan_2012_mon[jan_2012_mon.pickup_longitude<-72.5]
jan_2012_mon.shape

(2048831, 10)

jan_2012_mon contains all the Monday for January 2012

In [18]:
import matplotlib.path as mplPath

def indexZones(shapeFilename):
    import rtree
    import fiona.crs
    import geopandas as gpd
    index = rtree.Rtree()
    zones = gpd.read_file(shapeFilename).to_crs(fiona.crs.from_epsg(2263))
    for idx,geometry in enumerate(zones.geometry):
        index.insert(idx, geometry.bounds)
    return (index, zones)

def findBlock(p, index, zones):
    match = index.intersection((p.x, p.y, p.x, p.y))
    for idx in match:
        z = mplPath.Path(np.array(zones.geometry[idx].exterior))
        if z.contains_point(np.array(p)):
            return zones['OBJECTID'][idx]
    return -1

def mapToZone(parts):
    import pyproj
    import shapely.geometry as geom
    proj = pyproj.Proj(init="epsg:2263", preserve_units=True)    
    index, zones = indexZones("/Users/RAFAELLICHEN/Documents/CSC 59970/nyc-yellow-taxi-prediction/datasets/block-groups-polygons.geojson")
    i=0
    for line in parts:
        if (line["pickup_longitude"] and line["pickup_latitude"]):
            pickup_location  = geom.Point(proj(float(line["pickup_longitude"]), float(line["pickup_latitude"])))
            try:
                block_id_list.append(findBlock(pickup_location, index, zones))
            except AttributeError:
                drop_list.append(i)
        i+=1

drop_list=[]
block_id_list=[]

In [19]:
mapToZone(temp.T.to_dict().values())

In [28]:
print((len(drop_list)+len(block_id_list))==jan_2012_mon.shape[0])

True


In [51]:
jan_2012_mon=jan_2012_mon.drop(jan_2012_mon.index[drop_list])
jan_2012_mon["block_id"]=block_id_list
jan_2012_mon=jan_2012_mon.reset_index(drop=True)
jan_2012_mon=jan_2012_mon.loc[jan_2012_mon["block_id"]!=-1]
jan_2012_mon.head()

,pickup_datetime,dropoff_datetime,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,tip_amount,total_amount,weekday,block_id
0,2012-01-09 19:14:00,2012-01-09 19:20:00,1.25,-73.993335,40.727717,-73.981567,40.739245,0.00,7.60,0,9121
1,2012-01-09 19:12:00,2012-01-09 19:20:00,1.37,-73.975663,40.750380,-73.991625,40.738270,0.00,8.00,0,9423
2,2012-01-09 19:17:00,2012-01-09 19:21:00,0.71,-73.981810,40.778935,-73.981562,40.769470,1.00,7.40,0,2499
3,2012-01-09 19:02:00,2012-01-09 19:21:00,7.82,-73.885332,40.773002,-73.948070,40.774738,4.14,30.14,0,9608
4,2012-01-09 18:36:28,2012-01-09 18:41:58,1.00,-73.982000,40.774900,-73.973300,40.764300,1.00,7.80,0,9547


In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

x =data_2012_01["pickup_longitude"]
y =data_2012_01["pickup_latitude"]
z =data_2012_01["tip_amount"]

ax.scatter(x, y, z, c='r', marker='o')

ax.set_xlabel("longitude")
ax.set_ylabel("latitude")
ax.set_zlabel("tip amount")

plt.show()